## WELCOME TO WEEK 6

The Epic Finale Week

And

# WELCOME TO THE **M**ODEL **C**ONTEXT **P**ROTOCOL!

And welcome back to OpenAI Agents SDK ❤️❤️❤️

### Please note

There may be changes here from the video as I'm always making updates!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">To my Windows PC people - an important announcement</h2>
            <span style="color:#ff7800;">I have unpleasant news. There's a problem running MCP Servers on Windows PCs; Mac and Linux is fine. This is a known issue as of May 4th, 2025. I asked o3 with Deep Research to try to find workarounds; it <a href="https://chatgpt.com/share/6817bbc3-3d0c-8012-9b51-631842470628">confirmed the issue</a> and confirmed the workaround.<br/><br/>
            The workaround is a bit of a bore. It is to take advantage of "WSL", the Microsoft approach for running Linux on your PC. You'll need to carry out more setup instructions! But it's quick, and several students have confirmed that this works perfectly for them, then the Week 6 MCP labs work. Plus, WSL is actually a great way to build software on your Windows PC.<br/>
            The WSL Setup instructions are in the Setup folder, <a href="../setup/SETUP-WSL.md">in the file called SETUP-WSL.md here</a>. I do hope this only holds you up briefly - you should be back up and running quickly. Oh the joys of working with bleeding-edge technology!<br/><br/>
            With many thanks to students Markus, Abhi, Hui-Ling, and several others, for helping me work on it and confirming the fix.
            </span>
        </td>
    </tr>
</table>

# MCP Transport Mechanisms: `stdio` vs `sse`

MCP (Model Context Protocol) defines how a **host** (IDE, agent runtime, app) communicates with a **tool/server**. The protocol itself stays the same, but it supports different **transport mechanisms**. The two most common are **`stdio`** and **`sse`**.

---

### 1. `stdio` Transport

### What it is

* Communication over **standard input / standard output**
* MCP server runs as a **child process**
* Messages are exchanged as **JSON** via stdin/stdout

#### Mental model

> *"I run a tool as a local process and talk to it by sending JSON back and forth."*

This is similar to:

* Early **Language Server Protocol (LSP)** implementations
* CLI tools wrapped by other programs

#### How it works

1. Host launches the MCP server process
2. Host sends requests to **stdin**
3. Server writes responses/events to **stdout**
4. No networking involved

#### When it’s used

* Local tools
* IDE integrations
* Simple, secure environments

#### Pros

* ✅ Very simple setup
* ✅ No networking required
* ✅ Easy to sandbox
* ✅ Low overhead

#### Cons

* ❌ Local-only
* ❌ Not scalable
* ❌ Single host ↔ single server
* ❌ Hard to share across machines

---

### 2. `sse` Transport (Server-Sent Events)

#### What it is

* Communication over **HTTP**
* Uses **Server-Sent Events (SSE)** for streaming responses
* MCP server runs as a **long-lived HTTP service**

#### Mental model

> *"I send commands over HTTP and keep a stream open for real-time responses."*

Comparable to:

* Streaming chat responses
* Live log streaming
* Real-time dashboards

#### How it works

1. Host sends requests via HTTP
2. Server streams results and events using **SSE**
3. One server can serve multiple clients

#### When it’s used

* Remote or cloud-hosted tools
* Distributed agent systems
* Multi-client environments

#### Pros

* ✅ Network-accessible
* ✅ Scales well
* ✅ Supports multiple clients
* ✅ Cloud-friendly deployment

#### Cons

* ❌ More complex setup
* ❌ Requires HTTP server & auth
* ❌ Slightly higher latency than `stdio`

---

### 3. Side-by-Side Comparison

| Aspect       | `stdio`        | `sse`          |
| ------------ | -------------- | -------------- |
| Transport    | stdin / stdout | HTTP + SSE     |
| Network      | ❌ No           | ✅ Yes          |
| Setup        | Very simple    | More involved  |
| Deployment   | Local process  | Local or cloud |
| Scalability  | Low            | High           |
| Multi-client | ❌ No           | ✅ Yes          |

---

### 4. Key Insight

**The transport does NOT change MCP itself.**

* Same MCP message formats
* Same tools, prompts, and resources
* Only the **wire protocol** changes

This means you can:

* Develop locally using `stdio`
* Deploy remotely using `sse`
* Reuse the same MCP logic in both cases

In [1]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [2]:
load_dotenv(override=True)

True

### Let's use MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

Let's try the Fetch mcp-server that we looked at last week

### Special note - in case you updated the OpenAI Agents framework:

OpenAI Agents SDK recently upgraded their SDK and made a breaking change (thank you, OpenAI 😂). I haven't yet updated the version in this repo to the latest version. If you've upgraded the package yourself, then you might get an error about missing positional arguments. If you get that error, you'll need to replace `await server.list_tools()` with `await server.session.list_tools()` and change the last line from `fetch_tools` to `fetch_tools.tools`. And you'll need to make this change every time you get this error! The joys of working at the bleeding edge..

In [3]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

## Extra installation step - if you don't have Node and Playwright on your computer

The next MCP tool uses node (the Javascript Server), and it needs you to have the command 'npx' installed on your computer.

Extra setup steps are here:

[Node and Playwright setup](../setup/SETUP-node.md)

## And now 2 more MCP servers - this time Javascript based, with node

### If the next cell hangs, see the important troubleshooting at the end of the SETUP-node instructions linked above.

In [4]:

playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools


[Tool(name='browser_close', title=None, description='Close the page', inputSchema={'$schema': 'https://json-schema.org/draft/2020-12/schema', 'type': 'object', 'properties': {}, 'additionalProperties': False}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'$schema': 'https://json-schema.org/draft/2020-12/schema', 'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_c

In [5]:

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools

[Tool(name='read_file', title='Read File (Deprecated)', description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'description': 'If provided, returns only the last N lines of the file', 'type': 'number'}, 'head': {'description': 'If provided, returns only the first N lines of the file', 'type': 'number'}}, 'required': ['path']}, outputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'content': {'type': 'string'}}, 'required': ['content'], 'additionalProperties': False}, icons=None, annotations=ToolAnnotations(title=None, readOnlyHint=True, destructiveHint=None, idempotentHint=None, openWorldHint=None), meta=None, execution={'taskSupport': 'forbidden'}),
 Tool(name='read_text_file', title='Read Text File', description="Read the complete contents of a file from the

### And now.. bring on the Agent with Tools!

In [13]:
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled
import os

# BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
# API_KEY=os.getenv('GOOGLE_API_KEY')
# MODEL_NAME = "gemini-2.5-flash"
BASE_URL = "http://localhost:11434/v1"
client = AsyncOpenAI(base_url=BASE_URL)
MODEL_NAME = "qwen3:latest"
set_tracing_disabled(disabled=True)

instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""


async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model= OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client),
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for Chicken Fry Biryani, then summarize it in markdown to biryani.md")
            print(result.final_output)


The file `biryani.md` has been successfully created in the allowed `sandbox` directory. You can now access it via `/Users/himanshu/Documents/Projects/agentic-ai/6_mcp/sandbox/biryani.md`. 

Would you like to make any modifications to this file, or is there anything else I can assist with? 🥢米饭


#### Explanation about what exactly happens when we say MCP server is running (See 2nd chat onwards)
https://gemini.google.com/u/2/app/915dff6d40d31b55?is_sa=1&is_sa=1&android-min-version=301356232&ios-min-version=322.0&campaign_id=bkws&utm_source=sem&utm_medium=paid-media&utm_campaign=bkws&pt=9008&mt=8&ct=p-growth-sem-bkws&gclsrc=aw.ds&gad_source=1&gad_campaignid=22908443171&gbraid=0AAAAApk5Bhn586sDtpOAaLTzWP3ab9Mik&gclid=CjwKCAiAssfLBhBDEiwAcLpwfkxga6rXYgxFYnFmmvtxIoGVryS1vH6eo0BVRkcT3skSASiRbKbHcBoCz6sQAvD_BwE&pageId=none

### Check out the trace

https://platform.openai.com/traces

### Now take a look at some MCP marketplaces

https://mcp.so

https://glama.ai/mcp

https://smithery.ai/

https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

HuggingFace great community article:
https://huggingface.co/blog/Kseniase/mcp



